In [1]:
from _ctypes import LoadLibrary as _dlopen
_dlopen("D:\\Sangwons_Room\\00_SoftWares\\Anaconda\\envs\\torch\\Library\\bin\\geos.dll", 0)

140710694420480

In [2]:

# nuscenes-devkit
from nuscenes.nuscenes import NuScenes

# nuscenes-map expansion
from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.map_expansion import arcline_path_utils
from nuscenes.map_expansion.bitmap import BitMap

# nuscenes-devkit-utils
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.geometry_utils import view_points, transform_matrix

from pyquaternion import Quaternion


from functools import reduce
import os

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm
%matplotlib inline

In [3]:
nusc = NuScenes(version='v1.0-mini', dataroot='./data/sets/nuscenes', verbose=True)


Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.632 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [7]:
nusc_map = NuScenesMap(dataroot='./data/sets/nuscenes', map_name='singapore-onenorth')

In [4]:
class NusceneMapExpand(NuScenesMap):
    def __init__(self, dataroot = './data/sets/nuscenes', map_name = 'singapore-onenorth'):
        super().__init__(dataroot=dataroot, map_name = map_name)
            
    def get_closest_layers(self, layers, center_pose, patch = [1000, 1000], mode = 'within'):
        my_patch = [center_pose[0] - patch[0]/2, center_pose[1] - patch[1]/2, patch[0], patch[1]]
        records = self.get_records_in_patch(my_patch, layers, mode='within')
        output = dict()
        
        for layer in layers:
            if layer in self.non_geometric_polygon_layers:
                tokens = records[layer]
                layer_coords = []
                for token in tokens:
                    node_coords = self.get_polygon_bounds(layer, token)
                    layer_coords.append(node_coords)
                
            elif layer in self.non_geometric_line_layers:
                tokens = records[layer]
                layer_coords = []
                for token in tokens:
                    node_coords = self.get_line_bounds(layer, token)
                    layer_coords.append(node_coords)
            else:
                continue
            output[layer] = layer_coords
            
        return output
    
    def get_closest_structures(self, layers, center_pose, max_objs = 64, max_points = 30, patch = [1000, 1000], mode = 'within'):
        layer_dict = map_api.get_closest_layers(layers, center_pose, patch, mode)
        output_list = []
        for layer in layers:
            nodes_list = layer_dict[layer]
            for nodes in nodes_list:
                nodes = self.nodes_abstraction(nodes)
                struct_dict = {'class':layer, 'nodes':nodes}
                output_list.append(struct_dict)
                
        return output_list
        
    def nodes_abstraction(self, nodes):
        output = np.zeros((2, 30))
        length = nodes.shape[0]
        if length <= 30:
            output[:,:length] = nodes.transpose(1,0)
            
        
        return output
        
    def get_polygon_bounds(self, layer, token):
        
        record = self.get(layer, token)
        nodes = [self.get('node', token) for token in record['exterior_node_tokens']]
        node_coords = np.array([(node['x'], node['y']) for node in nodes])
        
        return node_coords
    
    def get_line_bounds(self, layer, token):
        
        record = self.get(layer, token)
        nodes = [self.get('node', token) for token in record['node_tokens']]
        node_coords = np.array([(node['x'], node['y']) for node in nodes])
        
        return node_coords
        
        
map_api = NusceneMapExpand('./data/sets/nuscenes', 'singapore-onenorth')

In [5]:
map_api = NusceneMapExpand(dataroot='./data/sets/nuscenes', map_name='singapore-onenorth')

In [6]:
result = map_api.get_closest_structures(['road_divider', 'road_block', 'walkway', 'traffic_light'], [600, 1000])
result

[{'class': 'road_divider',
  'nodes': array([[438.90493241, 442.41108213,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ],
         [935.13648026, 941.09236154,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
 